In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Stable Diffusion XL 1.0 - LoRA serving

<table align="left"><tbody><tr>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_pytorch_stable_diffusion_xl_lora.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_xl_lora.ipynb">
      <img src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" alt="GitHub logo"><br>
      View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates how to download the popular LoRA (Low-Rank Adaptation) adapters from huggingface.co or civitai.com, and serve it together with the [stabilityai/stable-diffusion-xl-base-1.0](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0) model on Vertex AI for online prediction.

### Objective

- Deploy the base model and the LoRA adapter to a [Vertex AI Endpoint resource](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for text-to-image.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Run the notebook

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. **[Optional]** [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

BUCKET_URI = "gs://"  # @param {type:"string"}

# @markdown 3. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

# @markdown 4. If you want to run predictions with A100 80GB or H100 GPUs, we recommend using the regions listed below. **NOTE:** Make sure you have associated quota in selected regions. Click the links to see your current quota for each GPU type: [Nvidia A100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_a100_80gb_gpus), [Nvidia H100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_h100_gpus).

# @markdown > | Machine Type | Accelerator Type | Recommended Regions |
# @markdown | ----------- | ----------- | ----------- |
# @markdown | a2-ultragpu-1g | 1 NVIDIA_A100_80GB | us-central1, us-east4, europe-west4, asia-southeast1, us-east4 |
# @markdown | a3-highgpu-2g | 2 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-4g | 4 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-8g | 8 NVIDIA_H100_80GB | us-central1, us-east5, europe-west4, us-west1, asia-southeast1 |

import base64
import glob
import importlib
import os
import sys
import uuid
from datetime import datetime
from io import BytesIO

from google.cloud import aiplatform, storage
from PIL import Image

! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

common_util = importlib.import_module(
    "vertex-ai-samples.community-content.vertex_model_garden.model_oss.notebook_util.common_util"
)

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# If a custom gcs bucket uri is not provided, a unique GCS bucket will be
# created for the purpose of this notebook.
now = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

# Create a unique GCS bucket for this notebook, if not specified by the user.
if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )

print(f"Using this GCS Bucket: {BUCKET_URI}")

# Set up the default SERVICE_ACCOUNT.
SERVICE_ACCOUNT = None
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print("Using this default Service Account:", SERVICE_ACCOUNT)

# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=PROJECT_ID)

# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/pytorch-inference.cu125.0-4.ubuntu2204.py310"


# Define common functions.
def base64_to_image(image_str):
    """Convert base64 encoded string to an image."""
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image


def image_grid(imgs, rows=2, cols=2):
    w, h = imgs[0].size
    grid = Image.new(
        mode="RGB", size=(cols * w + 10 * cols, rows * h), color=(255, 255, 255)
    )
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w + 10 * i, i // cols * h))
    return grid


def deploy_model(
    model_id, lora_id, accelerator_type, machine_type, accelerator_count=1
):
    common_util.check_quota(
        project_id=PROJECT_ID,
        region=REGION,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        is_for_training=False,
    )

    model_name = model_id
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "LORA_ID": lora_id,
        "TASK": "text-to-image-sdxl",
        "DEPLOY_SOURCE": "notebook",
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predict",
        serving_container_health_route="/health",
        serving_container_environment_variables=serving_env,
        model_garden_source_model_name="publishers/stability-ai/models/stable-diffusion-xl-base",
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
        system_labels={
            "NOTEBOOK_NAME": "model_garden_pytorch_stable_diffusion_xl_lora.ipynb"
        },
    )
    return model, endpoint


def get_bucket_and_blob_name(filepath):
    # The gcs path is of the form gs://<bucket-name>/<blob-name>
    gs_suffix = filepath.split("gs://", 1)[1]
    return tuple(gs_suffix.split("/", 1))


def upload_local_dir_to_gcs(local_dir_path, gcs_dir_path):
    """Uploads files in a local directory to a GCS directory."""
    client = storage.Client()
    bucket_name = gcs_dir_path.split("/")[2]
    bucket = client.get_bucket(bucket_name)
    for local_file in glob.glob(local_dir_path + "/**"):
        if not os.path.isfile(local_file):
            continue
        filename = local_file[1 + len(local_dir_path) :]
        gcs_file_path = os.path.join(gcs_dir_path, filename)
        _, blob_name = get_bucket_and_blob_name(gcs_file_path)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(local_file)
        print("Copied {} to {}.".format(local_file, gcs_file_path))

In [ ]:
# @title Upload and deploy model

# @markdown This section uploads the model to Model Registry and deploys it to a Vertex AI Endpoint resource. The model deployment step will take ~30 minutes to complete.

# @markdown In this example, we deploy the [stabilityai/stable-diffusion-xl-base-1.0](https://huggingface.co/stable-diffusion-xl-base-1.0) model together with a LoRA.

# @markdown Select one of the two preset LoRA options, or use your own LoRA.
# @markdown * `Huggingface`: [TheLastBen/Papercut_SDXL](https://huggingface.co/TheLastBen/Papercut_SDXL)
# @markdown * `Civitai`: [Pixel Art XL](https://civitai.com/models/120096/pixel-art-xl). This example downloads LoRA from Civitai, and uploads it to GCS bucket.
# @markdown * `Use your own`: Set your own lora in the next section.

PRESET_LORA = "Huggingface"  # @param ["Huggingface", "Civitai", "Use your own"]

# @markdown [Optional] If you selected `Use your own`, set your lora source in `CUSTOM_LORA`. It can be one of the following:
# @markdown * A [Hugging Face](https://huggingface.co) lora id
# @markdown * A GCS uri (starting with "gs://")
# @markdown * An http uri. In this case, we'll download the lora and upload it to GCS bucket.

CUSTOM_LORA = ""  # @param {type: "string"}

accelerator_type = "NVIDIA_L4"  # @param ["NVIDIA_L4", "NVIDIA_A100_80GB"]
machine_type_map = {
    "NVIDIA_L4": "g2-standard-8",
    "NVIDIA_A100_80GB": "a2-ultragpu-1g",
}

if PRESET_LORA != "Use your own" and CUSTOM_LORA != "":
    print(
        f'Warning: PRESET_LORA [{PRESET_LORA}] is selected. CUSTOM_LORA will not be used.\nIf you want to use CUSTOM_LORA, select "Use your own" in PRESET_LORA.\n'
    )

if PRESET_LORA == "Huggingface":
    lora_source = "TheLastBen/Papercut_SDXL"
elif PRESET_LORA == "Civitai":
    lora_source = "https://civitai.com/api/download/models/135931"
else:
    lora_source = CUSTOM_LORA

if lora_source.startswith("http"):
    # Download a LoRA adapter first and save it to a GCS bucket.
    ! rm -r /tmp/custom-lora
    ! mkdir /tmp/custom-lora

    destination_folder = "/tmp/custom-lora"
    file_name = "custom-lora.safetensors"

    target = f"{destination_folder}/{file_name}"

    !gdown --fuzzy -O $target "$lora_source"
    upload_local_dir_to_gcs("/tmp/custom-lora", f"{BUCKET_URI}/custom-lora")
    lora_id = f"{BUCKET_URI}/custom-lora"
else:
    # Juggingface id and gs uri can be used directly as `lora_id`
    lora_id = lora_source

model, endpoint = deploy_model(
    model_id="stabilityai/stable-diffusion-xl-base-1.0",
    lora_id=lora_id,
    accelerator_type=accelerator_type,
    machine_type=machine_type_map[accelerator_type],
)

In [ ]:
# @title Predict

# @markdown Once deployment succeeds, you can send a batch of text prompts to the endpoint to generated images.

# @markdown **Note:** Some LoRA adaptors require explicit prompt to activate. For
# @markdown example, the preset Hugging Face [TheLastBen/Papercut_SDXL](https://huggingface.co/TheLastBen/Papercut_SDXL)
# @markdown Lora requires an explicit prompt `"papercut"`. In this example,
# @markdown we'll automatically append the prompt for you.

# @markdown Example:

# @markdown ```
# @markdown Prompt: A serious capybara at work, wearing a suit
# @markdown ```

# @markdown You may adjust the parameters below to achieve best image quality.

prompt = "A serious capybara at work, wearing a suit"  # @param {type: "string"}
negative_prompt = ""  # @param {type: "string"}
height = 1024  # @param {type:"integer"}
width = 1024  # @param {type:"number"}
num_inference_steps = 25  # @param {type:"number"}
guidance_scale = 2.5  # @param {type:"number"}

if PRESET_LORA == "Huggingface" and "papercut" not in prompt:
    prompt = "papercut " + prompt
    print('Adding "papercut" to prompt')
    print("prompt:", prompt)

instances = [{"text": prompt}]
parameters = {
    "negative_prompt": negative_prompt,
    "height": height,
    "width": width,
    "num_inference_steps": num_inference_steps,
    "guidance_scale": guidance_scale,
}

response = endpoint.predict(instances=instances, parameters=parameters)
images = [
    base64_to_image(prediction.get("output")) for prediction in response.predictions
]
image_grid(images, rows=1)

In [ ]:
# @title Clean up resources

# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continouous charges that may incur.

try:
    # Undeploy model and delete endpoint.
    endpoint.delete(force=True)

    # Delete model.
    model.delete()

except Exception as e:
    print(e)

# Delete bucket.
delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME